In [5]:
import tensorflow as tf
import pandas as pd
from time import time
import numpy as np
from google.colab import drive
import numpy as np
import datetime
import copy
from sklearn import tree
from keras.models import Sequential
from keras.layers import LSTM, Dense
import tensorflow as tf
from tensorflow import keras
from keras.layers import TimeDistributed, Conv1D, MaxPool1D, Flatten, LSTM, Dense, AveragePooling1D, Dropout, Conv2D, MaxPool2D
from keras.models import Sequential
from keras.regularizers import l2
import sklearn.metrics as metrics

In [6]:
tf.test.is_gpu_available('GPU')

True

In [7]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [8]:
same_distribution = pd.read_feather('/content/drive/MyDrive/SOICT Data/df0.01.feather')
diff_1_distribution = pd.read_feather('/content/drive/MyDrive/SOICT Data/Bản sao của A_data1.feather')
#diff_2_distribution = pd.read_feather('/content/drive/MyDrive/SOICT Data/Bản sao của A_data2.feather')
test_distribution = pd.read_feather('/content/drive/MyDrive/SOICT Data/df0.99.feather')

In [9]:
NUM_FEATURE = 256
NUM_CLASSES = 3
PACKET_NUM = 20
client_lr = 1e-4
NUM_EPOCHS = 30
BATCH_SIZE = 16

In [10]:
same_distribution

,flow_id,0,1,2,3,4,5,6,7,8,...,247,248,249,250,251,252,253,254,255,Label
0,18684,20,3,3,0,1,1,23,3,3,...,0,0,0,0,0,0,0,0,0,0
1,18684,23,3,3,1,72,132,31,48,48,...,192,16,47,246,165,184,81,76,221,0
2,18684,23,3,3,2,35,91,253,67,149,...,223,56,249,221,241,23,139,167,19,0
3,18684,23,3,3,0,26,215,59,186,138,...,0,0,0,0,0,0,0,0,0,0
4,18684,23,3,3,0,34,240,27,29,148,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1775,224068,149,115,215,98,209,80,89,205,163,...,0,0,0,0,0,0,0,0,0,2
1776,224068,64,159,203,94,152,253,152,6,232,...,0,0,0,0,0,0,0,0,0,2
1777,224068,148,200,196,149,76,36,176,251,55,...,0,0,0,0,0,0,0,0,0,2
1778,224068,2,138,59,135,125,141,233,52,31,...,0,0,0,0,0,0,0,0,0,2


In [11]:
def data_processing(df, NUM_FEATURES):
   y_train = df['Label']
   flow_id = df['flow_id']

   df = df/255

   X_train = df.drop(['Label', 'flow_id'], axis=1)
   X_train = X_train.to_numpy()

   X_train = X_train.reshape(-1,20, NUM_FEATURES)
   y_train = y_train.to_numpy()

   y_train = y_train.reshape(-1,20)[:,-1]
   return X_train, y_train

In [12]:
x_same_flow_distribution, y_same_flow_distribution = data_processing(same_distribution, NUM_FEATURE)
x_diff_1_flow_distribution, y_diff_1_flow_distribution = data_processing(diff_1_distribution, NUM_FEATURE)
x_diff_2_flow_distribution, y_diff_2_flow_distribution = data_processing(diff_2_distribution, NUM_FEATURE)
x_test_flow_distribution, y_test_flow_distribution = data_processing(test_distribution, NUM_FEATURE)

In [13]:
def MultiClassTrAdaBoost(trans_S, Multi_trans_D_S, label_S, Multi_label_D_S, test, N, label_T):
    trans_A = list(Multi_trans_D_S.values())[0]
    if len(Multi_trans_D_S) == 1:
        pass
    else:
        for i in range(len(Multi_trans_D_S)-1):
            p = i + 1
            trans_A = np.concatenate((trans_A, list(Multi_trans_D_S.values())[p]), axis=0)
    label_A = list(Multi_label_D_S.values())[0]
    if len(Multi_label_D_S) == 1:
        pass
    else:
        for i in range(len(Multi_label_D_S)-1):
            p = i + 1
            label_A = np.concatenate((label_A, list(Multi_label_D_S.values())[p]), axis=0)

    trans_data = np.concatenate((trans_A, trans_S), axis=0)
    trans_label = np.concatenate((label_A, label_S), axis=0)

    row_A = trans_A.shape[0]
    row_S = trans_S.shape[0]
    row_T = test.shape[0]

    if N >= row_A:
        print('The maximum of iterations should be smaller than ', row_A)

    test_data = np.concatenate((trans_data, test), axis=0)
    test_label = np.concatenate((trans_label, label_T), axis=0)

    weights_A = np.ones([row_A, 1]) / row_A
    weights_S = np.ones([row_S, 1]) / row_S

    weights = np.concatenate((weights_A, weights_S), axis=0)

    alpha_S = 0.5 * np.log((1 + np.sqrt(2 * np.log(row_A / N))))

    alpha_T = np.zeros([1, N])
    result_label = np.ones([row_A + row_S + row_T, N])

    predict = np.zeros([row_T])
    predict1 = np.zeros([row_T])
    print ('params initial finished.')
    print('='*60)

    trans_data = np.asarray(trans_data, order='C')
    trans_label = np.asarray(trans_label, order='C')
    test_data = np.asarray(test_data, order='C')
    test_label = np.asarray(test_label, order='C')

    histories = []

    for i in range(N):
        weights = calculate_ratio_weight(weights)

        result_label[:, i], error_rate , Source_index, start, history, model = Multi_train_classifier(Multi_trans_D_S, label_S,trans_data, trans_label, test_data,test_label, weights,row_A,row_S)

        if error_rate <= 1e-10:
            N = i
            break
        alpha_T[0, i] = np.log((1 - error_rate) / error_rate) + np.log(NUM_CLASSES - 1)
        print('Iter {}-th result :'.format(i))
        print('The {}-th diff-distribution training dataset is chosen to transfer'.format(Source_index))
        print('error rate :', error_rate, '|| alpha_T :',alpha_T[0, i] )
        print('-'*60)
        C_t = 2 * (1 - error_rate)
        for j in range(row_S):
            weights[row_A + j] = weights[row_A + j] * np.exp(alpha_T[0, i] * np.abs(result_label[row_A + j, i] - label_S[j]))
        for j in range( len(list(Multi_trans_D_S.values())[Source_index]) ):
            loc = start + j
            weights[loc] =  C_t * weights[loc] * np.exp(-alpha_S * np.abs(result_label[loc, i] - label_A[loc]))
#         -- Thử nghiệm
        for j in range(row_T):
            predict1[j] = result_label[row_A + row_S + j, :][np.argmax(alpha_T[0, :])]
        acc= (label_T == predict1).sum() / row_T
        with open('acc.txt', 'a+') as f:
            f.write(str(acc) + ',')
        rp = metrics.classification_report(label_T, predict1)
        with open('log.txt', 'a+') as f:
            f.write("Round "+ str(i) + str(acc) + ',\n' + rp)
        print("Acc of predict", acc)
        print(rp)

    for i in range(row_T):
        predict[i] = result_label[row_A + row_S + i, :][np.argmax(alpha_T[0, :])]

    print("MultiSourceTrAdaBoost is done")
    print('='*60)
    print('The prediction labels of test data are :')
    print(predict)
    print(predict.shape)
    print("Acc of test")
    acc = (label_T == predict).sum() / row_T
    print(acc)
    acc= (label_T == predict).sum() / row_T
    print(metrics.classification_report(label_T, predict))
    current_time = datetime.datetime.now().strftime("%Y%m%d%H%M%S")

    model.save(f"model_accuracy_{acc:.4f}")
    log_file_path = f'{acc}_{current_time}.txt'

    with open(log_file_path, 'a+') as log_file:
        log_file.write(metrics.classification_report(label_T, predict))
    return predict

def softmax(z):
    """Compute softmax values for each sets of scores in x."""
    e_z = np.exp(z - np.max(z))
    return e_z / e_z.sum(axis=0)

def calculate_ratio_weight(weights):
    total = np.sum(weights)
    return np.asarray(weights / total, order='C')


def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def calculate_error_rate(model, test_data, test_label):
    predictions = model.predict(test_data)  # Dự đoán nhãn trên tập kiểm tra
    predictions = np.argmax(predictions, axis=1)
    error_count = np.sum(predictions != test_label)  # Đếm số lượng mẫu bị phân loại sai
    error_rate = error_count / len(test_label)
    return error_rate

def train_CNN(trans_data, trans_label, test_data, test_label, ratio_weight):
    model = Sequential()

    model.add(Conv2D(filters=32, kernel_size=(5, 5), padding='Same',
                     activation='relu', input_shape=(20, NUM_FEATURE,1)))
    model.add(Conv2D(filters=32, kernel_size=(5, 5), padding='Same',
                     activation='relu'))
    model.add(MaxPool2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(filters=64, kernel_size=(3, 3), padding='Same',
                     activation='relu'))
    model.add(Conv2D(filters=64, kernel_size=(3, 3), padding='Same',
                     activation='relu'))
    model.add(MaxPool2D(pool_size=(2, 2), strides=(2, 2)))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(256, activation="relu"))
    model.add(Dropout(0.5))
    model.add(Dense(NUM_CLASSES, activation="softmax"))

    model.compile(optimizer=tf.keras.optimizers.Adam(
    learning_rate=client_lr), loss=tf.keras.losses.SparseCategoricalCrossentropy(),

    metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])
    history = model.fit(trans_data, trans_label, epochs=NUM_EPOCHS, batch_size=BATCH_SIZE, verbose = 2,
                    use_multiprocessing=True, shuffle=True, sample_weight=ratio_weight[:, 0])

    predictions = model.predict(
    test_data, verbose=2, use_multiprocessing=True, batch_size=BATCH_SIZE)

    flow_pred = np.argmax(predictions, axis=-1)

    print(test_data.shape)
    print(test_label.shape)
    er_r= calculate_error_rate(model, test_data, test_label)


    return flow_pred, er_r, history, model

def Multi_train_classifier(Multi_trans_D_S,label_S, trans_data, trans_label, test_data, test_label, weights,row_A,row_S):
    _result_label = np.ones([len(test_data), len(Multi_trans_D_S)])
    error_record = []
    start_record = []
    start = 0
    for item in range(len(Multi_trans_D_S)):
        start_record.append(start)
        sub_dataset = list(Multi_trans_D_S.values())[item]
        data_dim = len(sub_dataset)
        _trans_data = np.concatenate((trans_data[start : start + data_dim], trans_data[row_A:row_A + row_S]), axis=0)
        _trans_label = np.concatenate((trans_label[start : start + data_dim], trans_label[row_A:row_A + row_S]), axis=0)
        _ratio_weight = np.concatenate((weights[start : start + data_dim], weights[row_A:row_A + row_S]), axis=0)
        _result_label[:, item], _error_rate, history, model = train_LSTM(_trans_data, _trans_label, test_data, test_label, _ratio_weight)
        _result_label[:, item] = _result_label[:, item].reshape(-1)
        start += data_dim

        error_record.append(_error_rate)
    error_record = np.array(error_record)
    print(error_record)
    classifier_index = np.random.choice(np.flatnonzero(error_record == error_record.min()))
    return _result_label[:,classifier_index], error_record[classifier_index], classifier_index,start_record[classifier_index], history, model

In [ ]:

Multi_trans_D_S = {
'trans_A_1' : x_diff_1_flow_distribution,
#'trans_A_2' : x_diff_2_flow_distribution,
}
Multi_label_D_S = {
'label_A_1' :  y_diff_1_flow_distribution,
#'label_A_2' :  y_diff_2_flow_distribution,
}
test = x_test_flow_distribution
label_T = y_test_flow_distribution
trans_S = x_same_flow_distribution
label_S = y_same_flow_distribution
N = 10
histories = MultiClassTrAdaBoost(trans_S, Multi_trans_D_S, label_S, Multi_label_D_S, test, N, label_T)

params initial finished.
Epoch 1/30
396/396 [==============================] - 13s 12ms/step - loss: 1.1966e-04 - sparse_categorical_accuracy: 0.4602
Epoch 2/30
396/396 [==============================] - 4s 11ms/step - loss: 1.1018e-04 - sparse_categorical_accuracy: 0.5964
Epoch 3/30
396/396 [==============================] - 5s 13ms/step - loss: 8.2567e-05 - sparse_categorical_accuracy: 0.7634
Epoch 4/30
396/396 [==============================] - 5s 12ms/step - loss: 5.7682e-05 - sparse_categorical_accuracy: 0.8490
Epoch 5/30
396/396 [==============================] - 5s 11ms/step - loss: 4.0695e-05 - sparse_categorical_accuracy: 0.8767
Epoch 6/30
396/396 [==============================] - 5s 13ms/step - loss: 3.7923e-05 - sparse_categorical_accuracy: 0.8829
Epoch 7/30
396/396 [==============================] - 5s 12ms/step - loss: 3.4640e-05 - sparse_categorical_accuracy: 0.8980
Epoch 8/30
396/396 [==============================] - 5s 12ms/step - loss: 3.0264e-05 - sparse_categorical